In [1]:
import sys
sys.path.append( '/anaconda/envs/py37_pytorch/lib/python3.7')
sys.path.append('/anaconda/envs/py37_pytorch/lib/python3.7/site-packages')

In [2]:
%load_ext autoreload
%autoreload 2

import glob
import logging
import numpy as np
from importlib import reload  # Not needed in Python 2
import torch
from torch import nn
import torch.nn.functional as F
from torchvision.models import ResNet
from torch.utils.data import DataLoader,Dataset


from dataset.ASdataset import AS_Data
from dataset.ASdataset_obs_train_input import AS_Data_obs

device = torch.device("cuda"  if torch.cuda.is_available() else "cpu")
reload(logging)
logging.basicConfig(level=logging.INFO,#控制台打印的日志级别
                    filename='logging.txt',
                    filemode='a',##模式，有w和a，w就是写模式，每次都会重新写日志，覆盖之前的日志
                    #a是追加模式，默认如果不写的话，就是追加模式
                    format=
                    '%(asctime)s : %(message)s',
                    )

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
'''
0:CO 1:NH3 2:SO2 3:SULF 4:voc_mean 5:p_mean 6:NO_mean
7:voc^2 8:voc^3 9:NO^2 10:NO^3 11:NO^4 12:NO^5 13:NO*voc 14:NO*voc^3 15:NO^2*voc 16:NO^4*voc 17:NO^5*voc 18:NO*NH3
19 NH3^2 20 NH3^3
'''
for filename in sorted(glob.glob('/AS_data/zeyuan_folder/concat_data/NORM_rest_EM_2015_*.npy')):
    print(filename)
    data = np.load(filename)
    print(data.shape)
    nh3 = data[:,1:2,:,:]
    voc = data[:,4:5,:,:]
    no = data[:,6:7,:,:]
    new_data = np.concatenate([data,voc*voc,voc*voc*voc,no*no,no*no*no,no*no*no*no,no*no*no*no*no,no*voc,
                              no*voc*voc*voc,no*no*voc,no*no*no*no*voc,no*no*no*no*no*voc,no*nh3,nh3*nh3,nh3*nh3*nh3],axis=1)
    print(new_data.shape)
    with open(filename.replace('/AS_data/zeyuan_folder/concat_data/','/AS_data/zeyuan_folder/concat_data/MULTI_'),'wb') as f:
        np.save(f,new_data)

/AS_data/zeyuan_folder/concat_data/NORM_rest_EM_2015_01_744_07_182_232.npy
(744, 7, 182, 232)
(744, 21, 182, 232)
/AS_data/zeyuan_folder/concat_data/NORM_rest_EM_2015_04_720_07_182_232.npy
(720, 7, 182, 232)
(720, 21, 182, 232)
/AS_data/zeyuan_folder/concat_data/NORM_rest_EM_2015_07_744_07_182_232.npy
(744, 7, 182, 232)
(744, 21, 182, 232)
/AS_data/zeyuan_folder/concat_data/NORM_rest_EM_2015_10_744_07_182_232.npy
(744, 7, 182, 232)
(744, 21, 182, 232)


In [5]:
for filename in sorted(glob.glob('/AS_data/zeyuan_folder/concat_data/MULTI_NORM_rest_EM_2015_*.npy')):
    print(filename)
    data = np.load(filename)
    print(data.shape)
    PM25 = data[:,[1,3,4,6,7,9,10,11,13,15,16,18,19,20,],:,:]
    print(PM25.shape)
    with open(filename.replace('/AS_data/zeyuan_folder/concat_data/MULTI_','/AS_data/zeyuan_folder/concat_data/MULTI_PM25_'),'wb') as f:
        np.save(f,PM25)

/AS_data/zeyuan_folder/concat_data/MULTI_NORM_rest_EM_2015_01_744_07_182_232.npy
(744, 21, 182, 232)
(744, 14, 182, 232)
/AS_data/zeyuan_folder/concat_data/MULTI_NORM_rest_EM_2015_04_720_07_182_232.npy
(720, 21, 182, 232)
(720, 14, 182, 232)
/AS_data/zeyuan_folder/concat_data/MULTI_NORM_rest_EM_2015_07_744_07_182_232.npy
(744, 21, 182, 232)
(744, 14, 182, 232)
/AS_data/zeyuan_folder/concat_data/MULTI_NORM_rest_EM_2015_10_744_07_182_232.npy
(744, 21, 182, 232)
(744, 14, 182, 232)


In [14]:
data = np.array([[i for i in range(10)],[i for i in range(10,20)]])
print(data)
new_data = data[:,list(range(1,2))+[4]]
print(new_data)

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]]
[[ 1  4  6]
 [11 14 16]]


In [3]:
results = []
with open('log/log_step1_multi.txt','r',encoding='utf-8') as f:
    for line in f:
        if "cur test loss L1" in line:
            r = line.strip().split()[-1].split(',')
            if len(r) == 4: results.append(r)

In [4]:
total = []
cur = 10000
for result in results:
    t = 0
    for r in result: t += float(r)
    total.append(t)
    if t <= cur:
        cur = t
        print("Current Best")
        print(result)
        print(t)
        print('*'*20)

Current Best
['5.03397', '5.9036584', '0.8786848', '0.64728564']
12.463598840000001
********************
Current Best
['4.9156027', '5.119788', '0.7304886', '0.56270117']
11.32858047
********************
Current Best
['5.170702', '4.756192', '0.6924123', '0.5435761']
11.1628824
********************
Current Best
['4.702267', '5.120251', '0.68198866', '0.51287544']
11.017382099999999
********************
Current Best
['4.9041977', '4.6192193', '0.6626822', '0.5018446']
10.687943800000001
********************
Current Best
['4.5152006', '4.2217393', '0.61663723', '0.48309252']
9.83666965
********************
Current Best
['4.5306478', '4.012095', '0.58384037', '0.43486804']
9.56145121
********************
Current Best
['4.428879', '4.0700064', '0.58900553', '0.42889202']
9.516782949999998
********************
Current Best
['4.495114', '3.961007', '0.5827649', '0.42053103']
9.45941693
********************
Current Best
['4.507752', '3.976908', '0.5438822', '0.42646888']
9.45501108
**********

In [5]:
pm, o, no, so = 100000,10000,100000,10000
for result in results:
    if float(result[0]) < pm:
        pm = float(result[0])
        print('Cur best PM25')
        print(result[0])
    if float(result[1]) < o:
        o = float(result[1])
        print('Cur best O3')
        print(result[1])
    if float(result[2]) < no:
        no = float(result[2])
        print('Cur best NO2')
        print(result[2])
    if float(result[3]) < so:
        so = float(result[3])
        print('Cur best SO2')
        print(result[3])

Cur best PM25
5.03397
Cur best O3
5.9036584
Cur best NO2
0.8786848
Cur best SO2
0.64728564
Cur best PM25
4.9156027
Cur best O3
5.119788
Cur best NO2
0.7304886
Cur best SO2
0.56270117
Cur best O3
4.756192
Cur best NO2
0.6924123
Cur best SO2
0.5435761
Cur best PM25
4.702267
Cur best NO2
0.68198866
Cur best SO2
0.51287544
Cur best O3
4.6192193
Cur best NO2
0.6626822
Cur best SO2
0.5018446
Cur best PM25
4.5152006
Cur best O3
4.2217393
Cur best NO2
0.61663723
Cur best SO2
0.48309252
Cur best NO2
0.58270866
Cur best PM25
4.420896
Cur best NO2
0.57561046
Cur best SO2
0.45747197
Cur best PM25
4.4197464
Cur best NO2
0.5568207
Cur best SO2
0.4569884
Cur best SO2
0.45609105
Cur best O3
4.1729364
Cur best O3
4.012095
Cur best SO2
0.43486804
Cur best SO2
0.42889202
Cur best PM25
4.40484
Cur best NO2
0.55420876
Cur best SO2
0.4261891
Cur best NO2
0.5260673
Cur best NO2
0.522208
Cur best SO2
0.4259766
Cur best NO2
0.5145278
Cur best SO2
0.42049304
Cur best O3
3.961007
Cur best SO2
0.41482252
Cur best